In [19]:
#importação das bibliotecas
import pandas as pd
import json
import unicodedata
import numpy as np

In [21]:
#Caminho dos arquivos 
path_vagas     = 'C:/Users/giuliasilva/Desktop/Estudo/POS/TC - Modulo 05/vagas.json'
path_prospects = 'C:/Users/giuliasilva/Desktop/Estudo/POS/TC - Modulo 05/prospects.json'
path_apps      = 'C:/Users/giuliasilva/Desktop/Estudo/POS/TC - Modulo 05/applicants.json'

#Subida dos dados para os dicionários
with open(path_vagas,     'r', encoding='utf-8') as f: vagas_dict     = json.load(f)
with open(path_prospects, 'r', encoding='utf-8') as f: prospects_dict = json.load(f)
with open(path_apps,      'r', encoding='utf-8') as f: apps_dict      = json.load(f)



In [ ]:
#DF de vagas existentes
vagas_list = []
for vaga_id, bloco in vagas_dict.items():
    registro = {'vaga_id': vaga_id}
    for chave in ['perfil_vaga', 'informacoes_basicas', 'beneficios']:
        registro.update(bloco.get(chave, {}))
    vagas_list.append(registro)

df_vagas = pd.DataFrame(vagas_list)



In [ ]:
#DF de profissionais, são os dados detalhados de todos eles 
apps_list = []

for codigo, bloco in apps_dict.items():
    registro = {'codigo_profissional': codigo}
    
    # Lista de seções esperadas no bloco do candidato
    secoes = [
        'infos_basicas',
        'informacoes_pessoais',
        'informacoes_profissionais',
        'formacao_e_idiomas'
    ]
    
    # Atualiza o registro com os dados disponíveis em cada seção
    for secao in secoes:
        registro.update(bloco.get(secao, {}))
    
    # Adiciona os campos de currículo (texto livre)
    registro['cv_pt'] = bloco.get('cv_pt', '') or ''
    registro['cv_en'] = bloco.get('cv_en', '') or ''
    
    apps_list.append(registro)

# Converte a lista em DataFrame
df_apps = pd.DataFrame(apps_list)


In [ ]:
#DF de candidatos por vaga
prospects_list = []

for vaga_id, bloco in prospects_dict.items():
    titulo = bloco.get('titulo')
    modalidade = bloco.get('modalidade')

    for prospect in bloco.get('prospects', []):
        registro = prospect.copy()
        registro.update({
            'vaga_id': vaga_id,
            'titulo_vaga': titulo,
            'modalidade_vaga': modalidade
        })
        prospects_list.append(registro)

df_prospects = pd.DataFrame(prospects_list)

In [ ]:
#Função para analise dos dados 
def analise_inicial(df, nome_df="DataFrame"):
    print(f"📊 Análise do {nome_df}")
    print("-" * 50)
    
    # Garante exibição completa de colunas
    pd.set_option('display.max_columns', None)
    
    # Dimensões
    print(f"Shape (linhas, colunas): {df.shape}\n")
    
    # Tipos de dados
    print("Tipos de dados por coluna:")
    print(df.dtypes)
    print("\n")

    # Valores nulos
    print("Valores ausentes por coluna:")
    display(df.isnull().sum().to_frame("Total Nulos").sort_values("Total Nulos", ascending=False))
    print("\n")

    # Restaura a configuração padrão após análise
    pd.reset_option('display.max_columns')
    
    print("\n" + "="*50 + "\n")


In [ ]:
#Consumo da função de analise de dados  inicial
analise_inicial(df_vagas, "df_vagas")
analise_inicial(df_apps, "df_apps")
analise_inicial(df_prospects, "df_prospects")

In [ ]:
#Função para analise de dados nulos em detalhes
def resumo_missing(df, nome_df="DataFrame", limite_descartar=50):
    print(f"Análise de valores ausentes - {nome_df}")
    print("-" * 60)
    
    total = df.isnull().sum()
    perc = (df.isnull().mean() * 100).round(2)
    
    resumo = pd.DataFrame({
        'Total Nulos': total,
        '% de Nulos': perc
    })
    
    # Apenas colunas com nulos
    resumo = resumo[resumo['Total Nulos'] > 0]
    
    if resumo.empty:
        print("Nenhum valor nulo encontrado!\n")
        return
    
    display(resumo.sort_values('% de Nulos', ascending=False))
    print("\n" + "="*60 + "\n")


In [ ]:
#consumo da função de analise de nulos detalhada
resumo_missing(df_vagas, "df_vagas")
resumo_missing(df_apps, "df_apps")
resumo_missing(df_prospects, "df_prospects")


In [ ]:
#antes das alterações acho que vale compreender o que são os dados e como substituir
#criarei um dicionário para conhecer melhor a base 

In [ ]:
def gerar_dicionario_estrutural(df, nome_df="DataFrame"):
    dic = pd.DataFrame({
        'nome_coluna': df.columns,
        'tipo_dado': df.dtypes.astype(str),
        'df_origem': nome_df,
        'descricao': '',  # Preencher manualmente
    })
    return dic


In [ ]:
dic_vagas = gerar_dicionario_estrutural(df_vagas, "df_vagas")
dic_prospects = gerar_dicionario_estrutural(df_prospects, "df_prospects")
dic_apps = gerar_dicionario_estrutural(df_apps, "df_apps")

In [ ]:
dic_vagas.head()

In [ ]:
# Cria DataFrames a partir de cada dicionário
df_dic_vagas = pd.DataFrame(dic_vagas)
df_dic_prospects = pd.DataFrame(dic_prospects)
df_dic_apps = pd.DataFrame(dic_apps)

# Define o caminho de destino
caminho = "C:/Users/giuliasilva/Desktop/Estudo/POS/TC - Modulo 05/application_web/data/"

# Exporta cada DataFrame para CSV
df_dic_vagas.to_csv(f"{caminho}dicionario_df_vagas.csv", index=False, encoding='utf-8-sig')
df_dic_prospects.to_csv(f"{caminho}dicionario_df_prospects.csv", index=False, encoding='utf-8-sig')
df_dic_apps.to_csv(f"{caminho}dicionario_df_apps.csv", index=False, encoding='utf-8-sig')


In [ ]:
#finalizei o dicionário e os campos que estão nulos seriam de bstante valor pra treinar o modelo, por exemplo: no df de vagas, habilidades comportamentais, no df_apps quaificações dos profissionais, experiencias....
#bem vamos seguir com  o  preenchimento de um dado  padrão "nan" para o que estiver vazio

In [ ]:
# Função para remover acentuação e caracteres especiais
def remove_acentos(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize('NFKD', texto)
        texto = ''.join(c for c in texto if not unicodedata.combining(c))
        texto = ''.join(c for c in texto if c.isalnum() or c.isspace())
        return texto
    return texto

# Função para normalizar texto
def normaliza_texto(x):
    if isinstance(x, str):
        x = remove_acentos(x)
        x = ' '.join(x.strip().lower().split())
        return x if x else np.nan
    return np.nan

# Aplica normalização em todo DataFrame, exceto datas
def normaliza_dataframe(df, colunas_data=[]):
    for col in df.columns:
        if col in colunas_data:
            # Converte data, mantém NaT se inválida ou nula
            df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
        else:
            df[col] = df[col].apply(normaliza_texto)
    return df


In [ ]:
# Colunas de data específicas
colunas_data_vagas = ['data_inicial', 'data_final', 'data_requicisao', 'prazo_contratacao']
colunas_data_apps = ['data_criacao', 'data_aceite', 'data_atualizacao', 'data_nascimento']
colunas_data_prospects = ['data_candidatura', 'ultima_atualizacao']

# Normalização
df_vagas = normaliza_dataframe(df_vagas, colunas_data_vagas)
df_apps = normaliza_dataframe(df_apps, colunas_data_apps)
df_prospects = normaliza_dataframe(df_prospects, colunas_data_prospects)


In [ ]:
#validando a conversão das datas 
print(df_vagas[colunas_data_vagas].dtypes)
print(df_apps[colunas_data_apps].dtypes)
print(df_prospects[colunas_data_prospects].dtypes)

In [ ]:
#consumo da função de analise de nulos detalhada para confirmar a mudança 
resumo_missing(df_vagas, "df_vagas")
resumo_missing(df_apps, "df_apps")
resumo_missing(df_prospects, "df_prospects")

In [ ]:
# Exibe apenas as linhas com pelo menos um valor nulo no df_vagas
df_vagas[df_vagas.isna().any(axis=1)]


In [ ]:
# Linhas com nulos no df_apps
df_apps[df_apps.isna().any(axis=1)]


In [ ]:
# Linhas com nulos no df_prospects
df_prospects[df_prospects.isna().any(axis=1)]


In [ ]:
print(df_apps.columns.tolist())


In [ ]:
df_apps.info()